In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# jax
import jax.numpy as jnp
from jax import random
import time

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm

# interpolation
from scipy.interpolate import CubicSpline

# pre-processing
from sklearn.preprocessing import StandardScaler

# collocation
import sys
import os
p_ = os.path.abspath(os.path.join('..', '00_utils'))
if p_ not in sys.path:
    sys.path.append(p_)
    
p_ = os.path.abspath(os.path.join('..', '00_models'))
if p_ not in sys.path:
    sys.path.append(p_)

import preprocess # helper preprocessing class

# pyomo
import importlib
import nn_jax_diffrax

To do: Interpolation can be done a single time before training & function saved


In [26]:
importlib.reload(nn_jax_diffrax)
NeuralODE_JAX = nn_jax_diffrax.NeuralODE

importlib.reload(preprocess)
DataPreprocessor = preprocess.DataPreprocessor

# , 'temperature': 'var1'
file_path = '../00_data/df_train.csv'
encoding = {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'}

layer_widths = [7, 65, 65, 1]
learning_rate = 1e-3
rng = random.PRNGKey(42)

To do: Interpolation can be done a single time before training & function saved


In [27]:
from datetime import datetime, timedelta

# prepare data ranges
START_DATE = '2015-01-10'
start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
date_sequences = [start_date + timedelta(days=i*5) for i in range(10)]
date_sequences_str = [date.strftime('%Y-%m-%d') for date in date_sequences]
print(date_sequences_str)

times_elapsed = []

['2015-01-10', '2015-01-15', '2015-01-20', '2015-01-25', '2015-01-30', '2015-02-04', '2015-02-09', '2015-02-14', '2015-02-19', '2015-02-24']


In [28]:
experiment_results = {}

for START_DATE in date_sequences_str:
    data_loader = DataPreprocessor(file_path, start_date = START_DATE, number_of_points = 400, n_days = 1, m = 1, 
                                   feature_encoding = encoding, split = 200, equally_spaced = True, 
                                   smooth = False, num_nodes_mult = 1)
    
    data_subsample = data_loader.load_data()
    df_train, df_test = data_loader.preprocess_data(data_subsample)

    node_model = NeuralODE_JAX(layer_widths, time_invariant=True)
    state = node_model.create_train_state(rng, learning_rate)

    ys = jnp.atleast_2d(jnp.array(df_train['y'])).T
    ts = jnp.array(df_train['t'])
    Xs = jnp.array(df_train.drop(columns=['y', 't']))
    extra_args = (Xs, ts)
    y0 = jnp.array(ys[0])

    start_time = time.time()
    state = node_model.train(state, ts[:] 
                            , ys[:], y0
                            , num_epochs = 1000
                            , extra_args = extra_args[:]
                            )
    end_time = time.time()
    
    y_train_pred = node_model.neural_ode(state.params, y0, ts, state, extra_args)
    
    experiment_results[START_DATE] = {}
    experiment_results[START_DATE]['times_elapsed'] = end_time - start_time
    
    experiment_results[START_DATE]['mae_diffrax'] = np.mean(np.abs(np.squeeze(y_train_pred) - np.squeeze(ys)))
    experiment_results[START_DATE]['mse_diffrax'] = np.mean(np.square(np.squeeze(y_train_pred) - np.squeeze(ys)))
    # -------------------------------------------- DIFFRAX NODE PREDICTION (TRAIN) --------------------
    plt.figure(figsize=(15, 7))
    plt.plot(ts, ys, 'r--', label='True Data')  
    plt.plot(ts, np.squeeze(y_train_pred), 'b' ,label='Predicted Data') 
    plt.xlabel('Time')
    plt.ylabel('u(t)')
    plt.title(f"Diffrax Neural ODE - Train Data")
    plt.legend(loc ="lower right")
    plt.grid(True)
    plt.savefig(f'plots/diffrax/diffrax_solver_train_{START_DATE}.png', format='png')  
    plt.close() 

    # -------------------------------------------- DIFFRAX NODE PREDICTION (TEST) --------------------
    ys_test = jnp.atleast_2d(jnp.array(df_test['y'])).T
    ts_test = jnp.array(df_test['t'])
    Xs_test = jnp.array(df_test.drop(columns=['y', 't']))
    extra_args_test = (Xs_test, ts_test)
    y0_test = jnp.array(ys_test[0])
    
    y_test_pred = node_model.neural_ode(state.params, y0_test, ts_test, state, extra_args_test)
    
    experiment_results[START_DATE]['mae_diffrax_test'] = np.mean(np.abs(np.squeeze(y_test_pred) - np.squeeze(ys_test)))
    experiment_results[START_DATE]['mse_diffrax_test'] = np.mean(np.square(np.squeeze(y_test_pred) - np.squeeze(ys_test)))
    
    plt.figure(figsize=(15, 7))
    plt.plot(ts_test, ys_test, 'r--', label='True Data (Test)')  
    plt.plot(ts_test, np.squeeze(y_test_pred), 'b' ,label='Predicted Data (Test)') 
    plt.xlabel('Time')
    plt.ylabel('u(t)')
    plt.title(f"Diffrax Neural ODE - Test Data")
    plt.legend(loc ="lower right")
    plt.grid(True)
    plt.savefig(f'plots/diffrax/diffrax_solver_test_{START_DATE}.png', format='png')  
    plt.close() 

Epoch 100, Loss: 0.8166288733482361
Epoch 200, Loss: 0.6926212310791016
Epoch 300, Loss: 0.6371097564697266
Epoch 400, Loss: 0.6148166656494141
Epoch 500, Loss: 0.5973230004310608
Epoch 600, Loss: 0.6010914444923401
Epoch 700, Loss: 0.5636014938354492
Epoch 800, Loss: 0.5443029403686523
Epoch 900, Loss: 0.5165137052536011
Epoch 1000, Loss: 0.4992322027683258
Epoch 100, Loss: 0.9511800408363342
Epoch 200, Loss: 0.7797123789787292
Epoch 300, Loss: 0.6828694343566895
Epoch 400, Loss: 0.5641154646873474
Epoch 500, Loss: 0.45781856775283813
Epoch 600, Loss: 0.38864433765411377
Epoch 700, Loss: 0.34057408571243286
Epoch 800, Loss: 0.30896252393722534
Epoch 900, Loss: 0.28064948320388794
Epoch 1000, Loss: 0.28121814131736755
Epoch 100, Loss: 0.9835340976715088
Epoch 200, Loss: 0.7612800002098083
Epoch 300, Loss: 0.6586185097694397
Epoch 400, Loss: 0.5834441781044006
Epoch 500, Loss: 0.5397061705589294
Epoch 600, Loss: 0.49830663204193115
Epoch 700, Loss: 0.46874141693115234
Epoch 800, Loss: 0

In [31]:
experiment_results['2015-01-10']

{'times_elapsed': 4.602709054946899,
 'mae_diffrax': 0.5773487,
 'mse_diffrax': 0.49768555,
 'mae_diffrax_test': 1.6517676,
 'mse_diffrax_test': 3.3748527}

In [32]:
print(f"MAE train: {np.mean([experiment_results[k]['mae_diffrax'] for k in experiment_results.keys()]):.3f}")
print(f"MAE test: {np.mean([experiment_results[k]['mae_diffrax_test'] for k in experiment_results.keys()]):.3}")
print(f"MSE train: {np.mean([experiment_results[k]['mse_diffrax'] for k in experiment_results.keys()]):.3f}")
print(f"MSE test: {np.mean([experiment_results[k]['mse_diffrax_test'] for k in experiment_results.keys()]):.3}")
print(f"Avg sovler time elapsed: {np.mean([experiment_results[k]['times_elapsed'] for k in experiment_results.keys()]):.3} seconds")

MAE train: 0.501
MAE test: 1.21
MSE train: 0.371
MSE test: 2.51
Avg sovler time elapsed: 5.15 seconds


In [7]:
print(f"{1e-5:.5f}")

0.00001
